## 모델 출력 확인하기
1. 모델 학습/평가 관리용 클래스인 ModelManager의 인스턴스를 생성합니다.
2. 학습/평가용으로 불러올 전처리 데이터의 구조는 `BaseDataset` 클래스로 구현됩니다.
3. `BaseDataset`은 `torch.utils.data.Dataset` 클래스를 상속받는데, 이를 통해 전처리 데이터를 `DataLoader`로 관리할 수 있게 됩니다.
4. `DataLoader`는 `BaseDataset`에 구현된 데이터 구조를 batch data 형태로 바꾸기 위해 데이터의 모든 요소에 차원을 하나 추가합니다.<br/>
   -> 상세: `BaseDataset`은 인스턴스 생성 시(즉 __init__함수에서) `list[dict[str, Tensor]]` 형태의 데이터를 생성하고 self.behaviors_parsed에 저장합니다.<br/>
   즉 데이터를 하나 뽑으면 `dict[str, Tensor]` 형태의 구조를 갖습니다.<br/>
   그런데 mini batch 학습을 위해서는 여러개의 데이터를 하나로 묶어서 batch data를 생성해야 하고, 이 기능을 `DataLoader`로 수행합니다.<br/>
   `DataLoader`에서는 여러 데이터를 하나로 묶어 `dict[str, list[Tensor]]` 형태로 변경합니다.<br/>
   `list[Tensor]`의 길이는 config파일의 batch_size로 정해집니다.<br/>
   여기서 1 epoch의 총 iteration은 behaviors의 총 데이터 수 / batch_size로, 배치 데이터의 총 개수와 같습니다.<br/>
5. 모든 모델 클래스가 상속 받는 `pl.LightningModule`의 구현 방식으로 인해, 모델의 인스턴스 자체를 함수처럼 사용하면 해당 클래스에 구현된 forward() 함수가 실행됩니다.
6. forward 함수는 학습 시 사용하는 batch data를 받아, behaviors의 사용자 history를 기반으로 해당 사용자의 impression 목록의 click probability를 예측하고 반환합니다.<br/>
   -> 상세: behaviors의 모든 데이터는 크게 유저의 history, 해당 유저의 impressions 데이터로 구성됩니다.<br/>
   여기서 history는 해당 유저가 과거에 열람한 뉴스 목록, impressions는 이러한 history를 가진 유저에게 특정 시점에 화면에 노출된 뉴스 목록입니다.<br/>
   여기서 impressions에는 유저가 해당 뉴스를 클릭했는지(1), 하지 않았는지(0)가 1과 0으로 라벨링 되어있습니다.<br/>
   즉 모델이 history만으로 impressions의 모든 뉴스 목록에 대해 해당 history를 가진 유저의 클릭 가능성을 예측하고, 라벨과 비교하거나 순위를 매겨보면 해당 모델이 추천을 얼마나 정확하게 하는지를 계산할 수 있습니다. 
7. 여기서 반환 형태는 Tensor인데, 내부 데이터는 list[list[float]] 형태입니다. 즉 입력한 모든 batch data에 대한 예측 결과가 반환되는 것입니다.<br/>
   -> 상세: 예를 들어 batch_size가 2라면, 각 배치마다 behaviors의 데이터가 2개씩 포함될 것입니다.<br/>
    따라서 예측해야할 유저와 impression 쌍도 두개이므로, 반환하는 결과 데이터도 2개입니다.

## 1. ckpt 파일로 모델 불러오기

In [1]:
import os
from os import path
import sys

PROJECT_DIR = path.abspath(path.join(os.getcwd(), "..", ".."))
sys.path.append(PROJECT_DIR)

from torch import Tensor
from utils.model_manager import ModelManager
from utils.base_manager import ManagerArgs

c:\Users\nclud\anaconda3\envs\newsrec\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
args = ManagerArgs(
    config_path = path.join(PROJECT_DIR, "config/model/nrms/exp_demo1.yaml"),
    test_ckpt_path = path.join(PROJECT_DIR, "logs/lightning_logs/checkpoints/nrms/exp_demo1/epoch=24-val_auc_epoch=0.6996.ckpt")
)

In [3]:
model_manager = ModelManager(PROJECT_DIR, args, "test")
model_manager.model.eval()

Seed set to 1234
100%|██████████| 7538/7538 [00:09<00:00, 823.61it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


NRMS(
  (news_encoder): TimeDistributed(
    (module): NewsEncoder(
      (word_embedding): Embedding(42561, 300, padding_idx=0)
      (mh_selfattention): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=300, out_features=300, bias=True)
      )
      (additive_attention): AdditiveAttention(
        (linear): Linear(in_features=300, out_features=200, bias=True)
      )
    )
  )
  (user_encoder): UserEncoder(
    (mh_selfattention): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=300, out_features=300, bias=True)
    )
    (additive_attention): AdditiveAttention(
      (linear): Linear(in_features=300, out_features=200, bias=True)
    )
  )
  (val_performance_metrics): MetricCollection(
    (val_auc): AUC()
    (val_mrr): MRR()
    (val_ndcg@10): NDCG()
    (val_ndcg@5): NDCG()
  )
  (val_sentiment_diversity_metrics_vader): MetricCollection(
    (val_senti@10_vader): Senti()
    (val_senti@5_vader): Senti()
    (val_s

## 2. 테스트용 데이터 불러오기

In [4]:
from torch.utils.data import DataLoader
import itertools

def get_batch_from_dataloader(dataloader: DataLoader, index: int):
    """
    DataLoader는 torch.utils.data.Dataset 클래스를 상속받아 정의된 데이터셋의 인스턴스를 받아,
    데이터를 설정한 batch size에 맞게 묶어준 뒤 iterator의 형태로 하나씩 뽑아쓸 수 있게 만들어져 있습니다.
    따라서 iter(dataloader)로 batch data를 하나씩 뽑아볼 수 있는 iterator를 생성하고,
    itertools로 index번째 데이터만 잘라내서 next()로 값을 뽑아내어 반환합니다.
    """
    iterator = iter(dataloader)
    item: dict = next(itertools.islice(iterator, index, index + 1))
    return item

def show_batch_struct(batch_data: dict):
    print(type(batch_data))
    print("{")
    for key in list(batch_data.keys()):
        value: Tensor = batch_data[key]
        items = value.tolist()
        inner_type = type(items[0]).__name__
        if inner_type == "list":
            inner_type = f"list[{type(items[0][0]).__name__}]"
            if inner_type == "list[list]":
                inner_type = f"list[list[{type(items[0][0][0]).__name__}]]"
        print(f"\t{key}:\ttype={type(value).__name__}, shape={tuple(value.shape)}, inner_type={inner_type}", end="")
        if inner_type == int:
            print(f", value:{items[0]}")
        else:
            print("")
    print("}")

In [ ]:
"""
모든 데이터의 첫 번째 차원의 shape 값은 batch size입니다.
즉 batch data 안에 실제로 어떤 데이터가 저장되어있는지 알아보기 위해 출력해볼 때
해당 값은 별로 의미가 없습니다. 

예를 들어 h_title의 shape 출력 결과의 각 숫자는 다음과 같은 의미를 지닙니다.
(batch_size, config파일에 설정한 max_history 값, 전처리 과정에서 설정한 max_title 값 = 제목의 최대 토큰 개수)

c_abstract은 다음과 같습니다.
(batch_size, 해당 impressions 데이터에 포함된 뉴스 개수, 전처리 과정에서 설정한 max_abstract 값 = 본문 요약의 최대 토큰 개수)
"""

batch_data = get_batch_from_dataloader(model_manager.test_loader, 0)
show_batch_struct(batch_data)

<class 'dict'>
{
	user:	type=Tensor, shape=(1,), inner_type=int
	h_title:	type=Tensor, shape=(1, 50, 20), inner_type=list[list[int]]
	h_abstract:	type=Tensor, shape=(1, 50, 50), inner_type=list[list[int]]
	h_category:	type=Tensor, shape=(1, 50), inner_type=list[int]
	h_subcategory:	type=Tensor, shape=(1, 50), inner_type=list[int]
	h_vader_sentiment:	type=Tensor, shape=(1, 50), inner_type=list[float]
	h_bert_sentiment:	type=Tensor, shape=(1, 50), inner_type=list[float]
	history_length:	type=Tensor, shape=(1,), inner_type=int
	c_title:	type=Tensor, shape=(1, 28, 20), inner_type=list[list[int]]
	c_abstract:	type=Tensor, shape=(1, 28, 50), inner_type=list[list[int]]
	c_category:	type=Tensor, shape=(1, 28), inner_type=list[int]
	c_subcategory:	type=Tensor, shape=(1, 28), inner_type=list[int]
	c_vader_sentiment:	type=Tensor, shape=(1, 28), inner_type=list[float]
	c_bert_sentiment:	type=Tensor, shape=(1, 28), inner_type=list[float]
	labels:	type=Tensor, shape=(1, 28), inner_type=list[int]
}
{

## 3. 모델에 batch data 입력하고 출력 확인하기

In [7]:
def get_result_by_batch(batch_data: dict):
    result: Tensor = model_manager.model(batch_data) # model.forward(batch_data) 와 동일하게 동작합니다.
    return result

def show_result_by_batch(batch_data: dict):
    result: Tensor = get_result_by_batch(batch_data)
    click_scores = result.tolist()[0]
    ranks = []
    for index, label in enumerate(batch_data["labels"].tolist()[0]):
        ranks.append([label, click_scores[index]])
    ranks.sort(key=lambda x: x[1], reverse=True)

    # 헤더 출력
    print(f"{'Rank':<6} {'Score':^10} {'Label':^6}")
    print("-" * 26)
    # 각 row 출력
    for rank, data in enumerate(ranks):
        label = data[0]
        score = data[1]
        print(f"{rank+1:<6} {score:^10.5f} {label:^6}")
    return result

def show_result_by_index(index):
    batch_data = get_batch_from_dataloader(model_manager.test_loader, index)
    result: Tensor = show_result_by_batch(batch_data)
    return result

"""
index를 바꿔서 원하는 데이터를 테스트해볼 수 있습니다.
"""
result = show_result_by_index(0)


Rank     Score    Label 
--------------------------
1       21.96449    0   
2       21.63984    0   
3       14.37055    0   
4       13.39139    0   
5       7.60423     0   
6       4.49830     0   
7       3.25125     0   
8       2.46181     0   
9       0.83084     0   
10      0.31242     0   
11      0.16820     0   
12      -0.67941    0   
13      -3.86770    1   
14      -5.43904    0   
15      -6.06323    0   
16      -8.16475    0   
17      -8.50863    0   
18      -8.97648    0   
19      -9.31507    0   
20     -10.63651    0   
21     -11.07257    0   
22     -12.26732    0   
23     -12.92856    0   
24     -12.96733    0   
25     -13.71577    0   
26     -17.14813    0   
27     -24.28173    0   
28     -29.99214    0   


In [8]:
print(result.data.shape)
print(result.shape)
print(result.tolist())

torch.Size([1, 28])
torch.Size([1, 28])
[[2.4618093967437744, 0.8308403491973877, -17.148134231567383, -12.928563117980957, -29.992136001586914, -8.508626937866211, -9.315069198608398, -24.281726837158203, 21.96449089050293, -3.8677048683166504, -8.976481437683105, 0.3124237358570099, -12.967334747314453, -12.267322540283203, -11.072565078735352, -10.636513710021973, 0.16820214688777924, 7.604233264923096, -6.063231468200684, -13.715768814086914, 14.370553016662598, 21.63984489440918, 13.391388893127441, 3.2512476444244385, -8.164746284484863, -5.439044952392578, 4.4983038902282715, -0.6794053316116333]]


In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# model.forward 가져오기
def forward(news_encoder, user_encoder, batch):
    # encode candidate news
    candidate_news_vector = news_encoder(batch["c_title"])
    
    # encode history 
    clicked_news_vector = news_encoder(batch["h_title"])
    # encode user
    user_vector = user_encoder(clicked_news_vector)
    # compute scores for each candidate news
    clicks_score = torch.bmm(
        candidate_news_vector,
        user_vector.unsqueeze(dim=-1)).squeeze(dim=-1)
    
    return clicks_score

In [67]:
print(torch.tensor([3.1, 2.4, 5.5, 6.6, 7.7]))
print(torch.tensor([3.1, 2.4, 5.5, 6.6, 7.7]).size())
print(torch.tensor([3.1, 2.4, 5.5, 6.6, 7.7]).shape)
print(F.dropout(torch.tensor([3.1, 2.4, 5.5, 6.6, 7.7]), p=0.5, training=True))

tensor([3.1000, 2.4000, 5.5000, 6.6000, 7.7000])
torch.Size([5])
torch.Size([5])
tensor([ 6.2000,  0.0000, 11.0000,  0.0000, 15.4000])


In [34]:
word_embedding = nn.Embedding.from_pretrained(
    model_manager.pretrained_word_embedding,
    freeze=model_manager.config.freeze_word_embeddings,
    padding_idx=0)

In [43]:
word_embedding(torch.tensor([0]))

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [11]:
news_encoder = model_manager.model.news_encoder
user_encoder = model_manager.model.user_encoder

forward(news_encoder, user_encoder, batch_data)

tensor([[  2.4618,   0.8308, -17.1481, -12.9286, -29.9921,  -8.5086,  -9.3151,
         -24.2817,  21.9645,  -3.8677,  -8.9765,   0.3124, -12.9673, -12.2673,
         -11.0726, -10.6365,   0.1682,   7.6042,  -6.0632, -13.7158,  14.3706,
          21.6398,  13.3914,   3.2512,  -8.1647,  -5.4390,   4.4983,  -0.6794]],
       device='cuda:0', grad_fn=<SqueezeBackward1>)

In [22]:
# encode candidate news
candidate_news_vector = news_encoder(batch_data["c_title"])

# encode history 
clicked_news_vector = news_encoder(batch_data["h_title"])
# encode user
user_vector = user_encoder(clicked_news_vector)

clicks_score = torch.bmm(
        candidate_news_vector,
        user_vector.unsqueeze(dim=-1)).squeeze(dim=-1)

In [24]:
print(candidate_news_vector.shape)
print(clicked_news_vector.shape)
print(user_vector.shape)
print(user_vector.unsqueeze(dim=-1).shape)
print(torch.bmm(
        candidate_news_vector,
        user_vector.unsqueeze(dim=-1)).shape)
print(clicks_score.shape)

torch.Size([1, 28, 300])
torch.Size([1, 50, 300])
torch.Size([1, 300])
torch.Size([1, 300, 1])
torch.Size([1, 28, 1])
torch.Size([1, 28])


In [26]:
torch.bmm(
        candidate_news_vector,
        user_vector.unsqueeze(dim=-1)).squeeze(-1)

tensor([[  2.4618,   0.8308, -17.1481, -12.9286, -29.9921,  -8.5086,  -9.3151,
         -24.2817,  21.9645,  -3.8677,  -8.9765,   0.3124, -12.9673, -12.2673,
         -11.0726, -10.6365,   0.1682,   7.6042,  -6.0632, -13.7158,  14.3706,
          21.6398,  13.3914,   3.2512,  -8.1647,  -5.4390,   4.4983,  -0.6794]],
       device='cuda:0', grad_fn=<SqueezeBackward1>)

In [31]:
print(batch_data['h_title'][0][20])

tensor([7264,   24, 4383,  893, 1741,  194,   22, 2301,  151, 5485,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0], device='cuda:0')
